In [3]:
import os
import xml
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
# Port interface dataset
# 
# This dataset contains 55 images in JPEG Format with 800x800 resolution
# Anotatations are stored in PASCAL VOC XML format
# Each image contains one or more objects

# Dataset info
DATASET_PATH = './content/image/'
LABELS = ['AUDIO', 'AUDIO_INPUT', 'AUDIO_OUTPUT', 'DISPLAYPORT', 'DVI', 'ETHERNET', 'HDMI', 'PARALLEL', 'POWER', 'PS', 'PS2', 'USB', 'USBC', 'VGA']
NUM_CLASSES = len(LABELS)
NUM_IMAGES = 55